In [ ]:
from rdflib import Namespace, Graph
from buildingmotif import BuildingMOTIF
from buildingmotif.namespaces import BRICK
from buildingmotif.dataclasses import Model, Library, Template
from buildingmotif.ingresses.xlsx import XLSXIngress
from buildingmotif.ingresses import TemplateIngressWithChooser, Record

In [ ]:
bm = BuildingMOTIF("sqlite://", "topquadrant") # in-memory

In [ ]:
BLDG = Namespace("urn:my_site/")
model = Model.create(BLDG)
_ = Library.load(ontology_graph="../libraries/brick/Brick.ttl")
lib = Library.load(directory="tutorial")

In [ ]:
tstat_templ = lib.get_template_by_name('my-thermostat')
point_templ = lib.get_template_by_name('my-tstat-points')

In [ ]:
def mapper(key: str) -> str:
    return {
        "tstat": "name",
        "name": "room",
        "temp_sensor": "temp",
        "setpoint": "sp",
        # 'co2' parameter has correct name in spreadsheet
    }.get(key, key)

def chooser(rec: Record) -> Template:
    if rec.rtype == 'thermostats':
        return tstat_templ
    elif rec.rtype == 'thermostat points':
        return point_templ.inline_dependencies()

In [ ]:
xlsx = XLSXIngress("tutorial/thermostats-with-points.xlsx")
ingress = TemplateIngressWithChooser(chooser, mapper, xlsx)

In [ ]:
model.add_graph(ingress.graph(BLDG))

In [ ]:
print(model.graph.serialize())